In [6]:
import redis

r = redis.Redis(host='localhost', port=6379, decode_responses=True)

Test Key:

In [7]:
# Example: Set and get a key
r.set('key', 'value')

True

In [8]:
print(r.get('key'))  # b'value'

value


In [10]:
# Add the media file metadata from the NFO file
metadata = {
    'plot': 'In 2071, roughly fifty years after an accident with a hyperspace gateway made the Earth almost uninhabitable, humanity has colonized most of the rocky planets and moons of the Solar System. Amid a rising crime rate, the Inter Solar System Police (ISSP) set up a legalized contract system, in which registered bounty hunters (also referred to as "Cowboys") chase criminals and bring them in alive in return for a reward.',
    'outline': 'In 2071, roughly fifty years after an accident with a hyperspace gateway made the Earth almost uninhabitable, humanity has colonized most of the rocky planets and moons of the Solar System. Amid a rising crime rate, the Inter Solar System Police (ISSP) set up a legalized contract system, in which registered bounty hunters (also referred to as "Cowboys") chase criminals and bring them in alive in return for a reward.',
    'lockdata': 'true',
    'dateadded': '2022-10-19 19:46:52',
    'title': 'Cowboy Bebop',
    'originaltitle': 'カウボーイビバップ',
    'writer': 'Hajime Yatate',
    'credits': 'Hajime Yatate',
    'trailer': 'plugin://plugin.video.youtube/?action=play_video&videoid=RI08P5SaJNU',
    'rating': '8.9',
    'year': '1998',
    'mpaa': 'TV-14',
    'imdb_id': 'tt0213338',
    'tmdbid': '30991',
    'premiered': '1998-04-02',
    'releasedate': '1998-04-02',
    'enddate': '1999-04-23',
    'runtime': '25',
    'genre': 'Western',
    'studio': 'WOWOW Prime',
    'tag': 'space western',
    'tvdbid': '76885',
    'art': '',
    'actor': '',
    'id': '76885',
    'episodeguide': '',
    'season': '-1',
    'episode': '-1',
    'status': 'Ended'
}

# Use hset with mapping to add the metadata
r.hset('media:cowboy_bebop', mapping=metadata)

0

In [11]:
# Retrieve metadata
print(r.hgetall('media:cowboy_bebop'))

{'dateadded': '2022-10-19 19:46:52', 'status': 'Ended', 'year': '1998', 'rating': '8.9', 'premiered': '1998-04-02', 'runtime': '25', 'trailer': 'plugin://plugin.video.youtube/?action=play_video&videoid=RI08P5SaJNU', 'tvdbid': '76885', 'lockdata': 'true', 'imdb_id': 'tt0213338', 'id': '76885', 'art': '', 'plot': 'In 2071, roughly fifty years after an accident with a hyperspace gateway made the Earth almost uninhabitable, humanity has colonized most of the rocky planets and moons of the Solar System. Amid a rising crime rate, the Inter Solar System Police (ISSP) set up a legalized contract system, in which registered bounty hunters (also referred to as "Cowboys") chase criminals and bring them in alive in return for a reward.', 'enddate': '1999-04-23', 'writer': 'Hajime Yatate', 'season': '-1', 'tag': 'space western', 'tmdbid': '30991', 'mpaa': 'TV-14', 'title': 'Cowboy Bebop', 'actor': '', 'episodeguide': '', 'genre': 'Western', 'credits': 'Hajime Yatate', 'studio': 'WOWOW Prime', 'epis

Script to process nfo

In [ ]:
import xml.etree.ElementTree as ET

# Load and parse the NFO file
tree = ET.parse('tvshow.nfo')
root = tree.getroot()

# Extract metadata from the NFO file
metadata = {}
for child in root:
    metadata[child.tag] = child.text

metadata